In [1]:
import gymnasium as gym
import torch
from skrl.envs.wrappers.torch import wrap_env
from skrl.memories.torch import RandomMemory
from skrl.trainers.torch import SequentialTrainer, ParallelTrainer
from skrl.utils import set_seed
from reward import simple_reward
from preprocess import preprocess
from lr_schedulers import CosineAnnealingWarmUpRestarts

set_seed(42)

gym.register(
    id="MultiDatasetDiscretedTradingEnv",
    entry_point="mc_env:MultiDatasetDiscretedTradingEnv",
    disable_env_checker=True,
)

[skrl:INFO] Seed: 42


In [2]:
env_cfg = dict(
    id="MultiDatasetDiscretedTradingEnv",
    dataset_dir="./data/train/month_15m/**/**/*.pkl",
    preprocess=preprocess,
    reward_function=simple_reward,
    positions=[-1, 1],
    multiplier=[2, 5, 20],
    trading_fees=0.01,
    borrow_interest_rate=0.03,
    portfolio_initial_value=1000,
    max_episode_duration="max",
    verbose=0,
    window_size=60,
)

In [3]:
obs = gym.make(**env_cfg).observation_space
env = gym.make_vec(
    vectorization_mode="sync",
    num_envs=32,
    **env_cfg,
)
env = wrap_env(env, wrapper="gymnasium")

[skrl:INFO] Environment wrapper: gymnasium


In [4]:
device = env.device
replay_buffer_size = 1024 * 8 * env.num_envs
memory_size = int(replay_buffer_size / env.num_envs)
memory = RandomMemory(memory_size=memory_size, num_envs=env.num_envs, device=device, replacement=False)

In [5]:
import torch.nn as nn


class DepthwiseSeparableConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(DepthwiseSeparableConv1d, self).__init__()
        self.depthwise = nn.Conv1d(
            in_channels, 
            in_channels, 
            kernel_size=kernel_size, 
            stride=stride, 
            padding=padding, 
            groups=in_channels  # Depthwise convolution
        )
        self.pointwise = nn.Conv1d(
            in_channels, 
            out_channels, 
            kernel_size=1  # Pointwise convolution
        )
        self.batchnorm = nn.BatchNorm1d(out_channels)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        x = self.batchnorm(x)
        x = self.activation(x)
        return x

In [6]:
import torch.nn as nn
from skrl.models.torch import DeterministicMixin, MultiCategoricalMixin, Model
from skrl.utils.spaces.torch import unflatten_tensorized_space


class Shared(MultiCategoricalMixin, DeterministicMixin, Model):
    def __init__(
        self,
        observation_space,
        action_space,
        device,
        clip_actions=False,
        unnormalized_log_prob=True,
        reduction="sum",
    ):
        Model.__init__(self, observation_space, action_space, device)
        MultiCategoricalMixin.__init__(self, unnormalized_log_prob, reduction)
        DeterministicMixin.__init__(self, clip_actions)

        self.net = nn.Sequential(
            nn.Conv1d(8, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv1d(64, 128, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.Flatten(),
             # torch.Size([batch, 1144])
            nn.Linear(896, 512),
            nn.BatchNorm1d(512),
            nn.ELU(),
            nn.Dropout(0.2),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ELU(),
            nn.Dropout(0.2),
            nn.Linear(128, 32),
            nn.BatchNorm1d(32),
            nn.ELU(),
        )

        self.mean_layer = nn.Linear(32, self.num_actions)
        self.log_std_parameter = nn.Parameter(torch.zeros(self.num_actions))

        self.value_layer = nn.Linear(32, 1)

    def act(self, inputs, role):
        if role == "policy":
            return MultiCategoricalMixin.act(self, inputs, role)
        elif role == "value":
            return DeterministicMixin.act(self, inputs, role)

    def compute(self, inputs, role):
        states = unflatten_tensorized_space(obs, inputs["states"]).permute(0, 2, 1)  # 32, 288, 7

        if role == "policy":
            self._shared_output = self.net(states)
            action = self.mean_layer(self._shared_output)
            return action, {}
        elif role == "value":
            shared_output = (
                self.net(states) if self._shared_output is None else self._shared_output
            ) # single forward-pass
            self._shared_output = None
            value = self.value_layer(shared_output)
            return value, {}

In [7]:
models = {}
models["policy"] = Shared(env.observation_space, env.action_space, device)
models["value"] = models["policy"]

for model in models.values():
    model.init_parameters(method_name="normal_", mean=0.0, std=0.1)

In [8]:
from skrl.agents.torch.ppo import PPO_DEFAULT_CONFIG

cfg = PPO_DEFAULT_CONFIG.copy()
cfg["rollouts"] = memory_size
cfg["learning_epochs"] = 64
cfg["mini_batches"] = 4
cfg["discount_factor"] = 0.99
cfg["learning_rate"] = 0
cfg["learning_starts"] = 0
cfg["learning_rate_scheduler"] = CosineAnnealingWarmUpRestarts
cfg["learning_rate_scheduler_kwargs"] = {
    "T_0": 16 * cfg["learning_epochs"],
    "T_mult": 2,
    "T_up": cfg["learning_epochs"],
    "eta_max": 1e-3,
    "gamma": 0.8,
}

cfg["experiment"]["write_interval"] = 1000 * 8
cfg["experiment"]["checkpoint_interval"] = 100000
cfg["experiment"]["directory"] = "runs/torch/mddt"

In [9]:
import warnings
from skrl.agents.torch.ppo import PPO

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

agent = PPO(
    models=models,
    memory=memory,
    cfg=cfg,
    observation_space=env.observation_space,
    action_space=env.action_space,
    device=device,
)
cfg_trainer = {"timesteps": 10000000, "headless": True, "environment_info": "pc_counter"}
trainer = ParallelTrainer(cfg=cfg_trainer, env=env, agents=[agent])
# agent.track_data("Episode/Position changed")

In [ ]:
trainer.train()

  1%|▏         | 148255/10000000 [09:14<6:22:36, 429.16it/s] 